## Introduction

This tutorial introduces Spark, a handy and powerful distributed programming framework for large-scale data processing. 

Let's first consider a practical problem: if you are given a text file, can you find out how many different words exist in that file? And what're their occurrences? This won't be a hard problem for you as a CMU student! You can simply write a program to read the file into memory, split the text to get a collection of words, and iterate over it to record different words and their occurrence, done! 

Then, what if you are given a corpus with many huge files that may contain millions of distinct words. Are you gonna to use the same way as described above to count the results? You probably cannot consider the amount of data you will need to read into memory and the data structure you need to maintain as well as the time it takes to do this job in a single machine. Naturally, one solution would be to run this job using many machines at the same time to make use of parallelism and utilize the resources provided by these machines. But here comes the problem, you will need to handle many problems under this distributed approach (e.g. task allocation, consistency, monitoring, machine communication...) which make this job no longer trivial.

Don't worry! The Spark programming framework I'll introduce in this tutorial is designed to deal with such problems for you as a programmer. It provides a clean interface and allows you to build a distributed program without worrying about all the complicated distributed programming details.

Spark is very useful in machine learning and data science in many different scenarios including feature engineering, model training, and model inferencing. Spark can be deployed in the large data center to speed up computing. It's easy to use and is very popular nowadays. It'll be of great help if you know this framework and can apply Spark in your daily work as a data science practitioner.

### Tutorial Content
To write a spark program, the programming language you can choose from including [scala](https://www.scala-lang.org/), [java](http://www.oracle.com/technetwork/java/javase/downloads/index.html), [python](https://www.python.org/) and [R](https://www.r-project.org/). This tutorial will use python to demonstrate how spark works.

The remaining part of this tutorial is organized into the following sections:

- [Environment setup and configuration](#Environment-setup-and-configuration)
- [Spark programming framework](#Spark-programming-framework)
- [Rewrite Feature Generating Problem in HW3](#Rewrite-Feature-Generating-Problem-in-HW3)
- [Where to go from here](#Where-to-go-from-here)

### Environment setup and configuration

Before writing any code, you will need to have Spark installed. Normally, Spark is configured in large computer clusters but it also supports running locally for the purpose of learning and debugging. So in this tutorial, you will configure Spark on your own PC and run it locally to learn how to use Spark and there is no difference between writing a Spark program that runs in a cluster or runs locally. However, all the distributed operations are actually running on your single machine while running locally.

#### Download Spark

First, follow the instructions and download [Spark](https://spark.apache.org/downloads.html). I would recommand using pip to install Spark if you could:

    $ pip install pyspark
    
Pyspark is the Spark library for python, this command will download pyspark and automatically configure the environment. Also, you are free to download the compressed package. Since Spark use the Hadoop Distributed File System as the underlying file system, it will need the support from [Hadoop](http://hadoop.apache.org/). You can download a pre-built package with Hadoop included or you can configure Spark with your own Hadoop.

#### Issue related to java

Spark runs on java virtual machine, so java must be installed. Although the latest version of Spark has the support for java 9, I observed some errors while running Spark with java 9. So I would recommend you to use [java 8](http://www.oracle.com/technetwork/java/javase/downloads/index.html) to run Spark. If you already have java 9 installed, you can first download java 8 and then use [jenv](http://www.jenv.be/) to configure multiple versions of java on your system and switch to java 8 while running Spark.

### Spark programming framework

In this section, I'll introduce the basic idea about Spark programming framework. 

First, let's walking through a simple Spark program.

In [1]:
import pyspark # Import the pyspark module

#### Resilient Distributed Dataset

A Spark cluster consists of a master node and several slave nodes. There is one driver program running on the master node which can be regarded as the program entry point. It will create the first RDD, invoke operations on it and collect results from cluster. The driver program is what you are going to write when developing a distributed program in Spark.

The programming interface provided by Spark is called *Resilient Distributed Dataset (RDD)*, it's an in-memory data representation that store data across multiple nodes in one cluster. To understand RDD, you can think it as a python list and each element in RDD is a list entry which is also known as a record. When you invoke an operation on RDD, it will perform on each record of this RDD, which is similar to the apply method of pandas dataframe. To create an RDD, you can either invoke the *parallelize()* method that transform an existing python collection into RDD or read data into the driver program from external sources (e.g. files, database) as RDD:

In [2]:
# 1. Initialize sparkContext, configuration parameters can be set at this time using SparkConf.
# Or you can configure through a configuration file.
sc_conf = pyspark.SparkConf()
sc = pyspark.SparkContext(conf=sc_conf)
# 2. Load data into Spark program. Imagine the words are read from a corpus :-)
word_list = ['Data', 'Science', 'Science', 'Practical', 'Data', 'Fun', 'is', 'Fun']
# 3. Using parallelize to transfer word_list into a spark RDD.
word_list_rdd = sc.parallelize(word_list)

#### Transformation & Action
The RDD programming interface provides two type of operations: **transformation** and **action**.

- Transformation will generate a new RDD based on the one that invokes this transformation. **Transformations are lazy**: it won't compute right away. Instead, it will record the transformation applied to this RDD and the actual computation will only happen when an action is invoked:

In [3]:
# 4. Map & Reduce "transformation" to get the count of different words in the input.
# No actual computations are actually happened at this point.
word_list_key_value_rdd = word_list_rdd.map(lambda x: (x, 1))
result_rdd = word_list_key_value_rdd.reduceByKey(lambda x, y: x + y)

Due to the narrow programming interface provided by Spark, we cannot directly use *word_list_rdd* to count the occurrences of different words. So, the above code creates two RDD by invoking *map()* and *reduceByKey()* function on existing RDD to get the counting result. It can be demonstrated with the following figure:![https://raw.githubusercontent.com/Albertle/15688_Tutorial/master/figure/First_Example.jpg](https://raw.githubusercontent.com/Albertle/15688_Tutorial/master/figure/First_Example.jpg)
1. First, the *map* function map each record in *word_list_rdd* to a key-value tuple of the form (word, 1), where word is the key with value 1.
2. Then, *reduceByKey()* will perform operation on all records with the same key. And here, it just add up the value of all record tuples with the same key to get the count of each words.

The *lambda* expression passed into these two operations can be regarded as a function. More about [lambda expression](http://www.secnetix.de/olli/Python/lambda_functions.hawk) if you are interested.

- Action will cause all transformation related to an RDD to actually compute and bring the results back to the driver program: 

In [4]:
# 5. Collect "action" to actually run all the transformation related to result_rdd before
# and bring the result to master.
result = result_rdd.collect()
print(result)
# 6. Stop the sparkContext created before.
sc.stop()

[('Science', 2), ('Practical', 1), ('Fun', 2), ('is', 1), ('Data', 2)]


![https://raw.githubusercontent.com/Albertle/15688_Tutorial/master/figure/result_1.jpeg](https://raw.githubusercontent.com/Albertle/15688_Tutorial/master/figure/result_1.jpeg)

#### Partitions
One core idea of Spark is partition. Let's take a look at the following code snippet to understand what is the partition of RDD. Below is a simple crawler program that get the html page of all address in the *crawler_address.txt* which are:
- https://www.cs.cmu.edu/~15719/index.html
- https://www.cmu.edu/
- http://store.steampowered.com/
- http://www.datasciencecourse.org/
- http://www.bbc.com/
- https://www.cmu.edu/ini/

In [5]:
import requests

# Crawler function that receive a url, and return the content of the webpage.
def crawler(address):
    res = requests.get(address)
    if res.status_code != 200:
        return ''
    else:
        return res.text

# 1. Initialize sparkContext and pass in default configuration.
sc = pyspark.SparkContext(conf=pyspark.SparkConf())
# 2. Read the file into Spark as an RDD. Each record of this RDD is a line in the file.
# The number 3 tell Spark to divide this RDD into 3 partitions, and each contains 2 url.
address_rdd = sc.textFile("crawler_address.txt", 3)
# 3. "Map" each http address into the corresponding html page.
crawler_result_rdd = address_rdd.map(crawler)
# 4. Trigger all computations and bring result back to driver program.
final_res = crawler_result_rdd.collect()
print(len(final_res)) # Output: 6
print(final_res) # Output: a list containing the web content of each page
sc.stop()

6
['<?xml version="1.0" encoding="UTF-8"?>\n<!DOCTYPE html \n PUBLIC "-//W3C//DTD XHTML 1.0 Strict//EN"\n "http://www.w3.org/TR/xhtml1/DTD/xhtml1-strict.dtd">\n<html xml:lang="en" lang="en">\n<head>\n<title>15-719 Advanced Cloud Computing (Spring 2018)</title>\n<link rel="stylesheet" type="text/css" href="style.css" />\n<meta http-equiv="Content-Script-Type" content="text/javascript" />\n<script type="text/javascript" src="/util.js">\n</script>\n<script>\n  (function(i,s,o,g,r,a,m){i[\'GoogleAnalyticsObject\']=r;i[r]=i[r]||function(){\n  (i[r].q=i[r].q||[]).push(arguments)},i[r].l=1*new Date();a=s.createElement(o),\n  m=s.getElementsByTagName(o)[0];a.async=1;a.src=g;m.parentNode.insertBefore(a,m)\n  })(window,document,\'script\',\'//www.google-analytics.com/analytics.js\',\'ga\');\n\n  ga(\'create\', \'UA-28140341-2\', \'cmu.edu\');\n  ga(\'send\', \'pageview\');\n\n</script>\n</head>\n<body>\n<h1>15-719 Advanced Cloud Computing (Spring 2018)</h1>\n<div id="topnavbar">\n<ul class="topn

![https://raw.githubusercontent.com/Albertle/15688_Tutorial/master/figure/result_2.jpeg](https://raw.githubusercontent.com/Albertle/15688_Tutorial/master/figure/result_2.jpeg)

You may observe there are some slight differences between this snippet and the previous one:

1. Here, *textFile()* is used to create the very first RDD. It creates an RDD by loading a file, and each file line is a record in the generated RDD.
2. *textFile()* has a second argument and is passed in 3. This second argument is the **partition number** and tells Spark to break down this RDD into 3 partitions, each partition contains two URLs. Each partition can be executed on different machines in parallel and this is why Spark can execute some very large tasks in a timely efficient manner. How many partitions an RDD should have is totally up to you. However, you need to make sure enough Spark slave is set up to execute these partitions, or parallelism won't be fully utilized. Do note that there are many other transformations take partition number as an optional argument.
3. The input argument of *map()* is the function *crawler*, which takes one URL as argument. *Map* function will pass each record in each partition of this RDD into *crawler()* to get the web content.

#### Lineage of RDD & Caching
The lineage of RDD is the key to ensure the lazy computation of transformations. In the first example, the lineage of RDD can be depicted as:

    word_list_rdd <==== word_list_key_value_rdd <=== result_rdd <==== collect() (The direction of arrow indicate dependency.)

This lineage is built when transformations are invoked, but actual computation doesn't happen. And when an action is invoked on *result_rdd*, the transformation chain is then actually computed to create corresponding RDDs.

There is one caveat here: when an RDD is computed in one lineage chain and is used by the following transformation, this RDD will **not** be stored in memory or write to disk by default. Instead, if you want to store a computed RDD, you need to explicitly invoke *persist()* or *cache()*. The following example demonstrates how to cache an RDD:

In [6]:
# Similar to the first example, except it perform one more action on word_list_key_value_rdd
from pyspark import StorageLevel
sc_conf = pyspark.SparkConf()
sc = pyspark.SparkContext(conf=sc_conf)
word_list = ['Data', 'Science', 'Science', 'Practical', 'Data', 'Fun', 'is', 'Fun']
word_list_rdd = sc.parallelize(word_list, 4)
word_list_key_value_rdd = word_list_rdd.map(lambda x: (x, 1))
# 1. Persist this RDD after it's computed.
word_list_key_value_rdd.persist(StorageLevel.MEMORY_AND_DISK)
result_rdd = word_list_key_value_rdd.reduceByKey(lambda x, y: x + y)
# 2. Actual computation happen.
result = result_rdd.collect()
print(result)
# 3. Here saveAsTextFile is another “action”, which will save this rdd into
# local file system, where each rdd entry will be a file line.
# Note: you need to make sure the folder 'key_value_output' does not already exist
# in this working directory.
word_list_key_value_rdd.saveAsTextFile('key_value_output')
# 4. Unpersist this RDD when no other computation is depend on it.
word_list_key_value_rdd.unpersist()
sc.stop()

[('Science', 2), ('Practical', 1), ('Fun', 2), ('is', 1), ('Data', 2)]


After executing *saveAsTextFile*, a folder named *key_value_output* will be created in your local file system, and five files will be created: ![https://raw.githubusercontent.com/Albertle/15688_Tutorial/master/figure/Third_Example.jpg](https://raw.githubusercontent.com/Albertle/15688_Tutorial/master/figure/Third_Example.jpg)
Each part-xxxxx file contains 2 records of this RDD.

In this case, there are actually two lineage chain:
    
    word_list_rdd <==== word_list_key_value_rdd <=== result_rdd <==== collect()
    word_list_rdd <==== word_list_key_value_rdd <=== saveAsTextFile()
So there are two dependencies upon *word_list_key_value_rdd*. When *collect()* is invoked on *result_rdd*, *word_list_key_value_rdd* is computed due to the first lineage. After that, another action *saveAsTextFile* is also depend on *word_list_key_value_rdd*, and by default, this RDD is not stored which means Spark will compute it again. 

However, in the code we invoked *persist(StorageLevel.MEMORY_AND_DISK)* on *word_list_key_value_rdd* which will explicitly make Spark to store this RDD after it's computed, so reuse of this RDD won't cause the same RDD to be computed again. There are several level of persistence which can be found [here](https://spark.apache.org/docs/latest/rdd-programming-guide.html#rdd-persistence), and *cache()* is equivalent to *persist(StorageLevel.MEMORY_ONLY)*

Since transformations are lazy, persist operation is also lazy. It will persist an RDD accordingly when the RDD is actually computed. So invocation of *persist()* on an RDD should be done before its computation happen. On the contrary, *unpersist()*, which will free the space used to store an RDD, will be executed at the point it's invoked.

#### Failure Recovery

The lineage of RDD also ensures the robustness of Spark. When failure happens and some computed RDDs are lost, Spark can recompute them automatically with the lineage information.

### Rewrite Feature Generating Problem in HW3

In the linear regression problem of HW3, we try to predict the time when a bus will arrive at a bus stop. The very first step we did is to do a basic feature engineering of the bus data. In this section, I will show how to rewrite the HW problem into a distributed Spark program.

I store part of (about 70000+ records, due to the submission limit) the trip data into *labeled_vdf.csv* file. For your convenience, the data has the format: ![https://raw.githubusercontent.com/Albertle/15688_Tutorial/master/figure/Fourth_Example.jpeg](https://raw.githubusercontent.com/Albertle/15688_Tutorial/master/figure/Fourth_Example.jpeg)

In [7]:
import csv
import math
from datetime import datetime

First, the program will read the csv file to load data in.

In [8]:
source_data_list = list()
header = list()
# 1. Load data into program. Each line will be read as a dictionary of fields and appended to a list.
with open('labeled_vdf.csv', 'r') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    header = csv_reader.__next__()
    for row in csv_reader:
        row_dict = dict()
        for i in range(0, len(header)):
            row_dict[header[i]] = row[i]
        source_data_list.append(row_dict)
print(len(source_data_list))
print(source_data_list[0])

70106
{'tmstmp': '8/11/16 16:11', 'vid': '3202', 'lat': '40.44067764', 'lon': '-80.00022125', 'hdg': '114', 'pid': '4521', 'rt': '61A', 'des': 'Swissvale', 'pdist': '118', 'spd': '18', 'tablockid': '061A-178', 'tatripid': '6759', 'eta': '27'}


![https://raw.githubusercontent.com/Albertle/15688_Tutorial/master/figure/result_3.jpeg](https://raw.githubusercontent.com/Albertle/15688_Tutorial/master/figure/result_3.jpeg)

Then, write the function that generates feture for each of the record passed in.

In [9]:
# 2. Main logic of map function.
# Input is a record line, which is a dictionary of fields.
# Return will also be a dictionary of generated features.
def generate_feature(record):
    feature_dict = dict()
    # a. Retain the numerical feature
    numerical_feature = ['pdist', 'spd', 'lat', 'lon', 'eta']
    for name in numerical_feature:
        feature_dict[name] = record[name]
    # b. Adding bias
    feature_dict['bias'] = 1
    # c. Create time format
    pi_2 = 2 * math.pi
    date = datetime.strptime(record['tmstmp'], '%m/%d/%y %H:%M')
    week_day = date.weekday() / 7
    feature_dict['sin_day_of_week'] = math.sin(week_day * pi_2)
    feature_dict['cos_day_of_week'] = math.cos(week_day * pi_2)
    
    hour = date.hour / 24
    feature_dict['sin_hour_of_day'] = math.sin(hour * pi_2)
    feature_dict['cos_hour_of_day'] = math.cos(hour * pi_2)
    
    time = (date.hour * 60 + date.minute) / (60 * 24)
    feature_dict['sin_time_of_day'] = math.sin(time * pi_2)
    feature_dict['cos_time_of_day'] = math.cos(time * pi_2)
    
    # d. Heading
    hdg = float(record['hdg']) / 360
    feature_dict['sin_hdg'] = math.sin(hdg * pi_2)
    feature_dict['cos_hdg'] = math.cos(hdg * pi_2)
    
    # e. Weekday
    if date.weekday() < 5:
        feature_dict['weekday'] = 1
    else:
        feature_dict['weekday'] = 0
    
    # f. Destination
    destination_feature = ['Braddock', 'Downtown', 'Greenfield Only', 'McKeesport', 'Murray-Waterfront', 'Swissvale']
    for des in destination_feature:
        if record['des'] == des:
            feature_dict[des] = 1
        else:
            feature_dict[des] = 0

    # g. Round
    rt_feature = ['61A', '61B', '61C', '61D']
    for rt in rt_feature:
        if record['rt'] == rt:
            feature_dict[rt] = 1
        else:
            feature_dict[rt] = 0
    return feature_dict    

Finally, the main Spark program to do feature engineering!

In [10]:
sc_conf = pyspark.SparkConf()
sc = pyspark.SparkContext(conf=sc_conf)
# Transform into Spark RDD with 10 partitions.
source_rdd = sc.parallelize(source_data_list, 10)
# Invoke map to generate feature for each RDD record.
feature_rdd = source_rdd.map(generate_feature)
# Invoke action and bring result back to driver program.
res = feature_rdd.collect()
print(len(res))
print(res[0])
print(res[0].keys())
# Save feature to csv file
with open('feature.csv', 'w') as output:
    writer = csv.DictWriter(output, res[0].keys())
    writer.writeheader()
    for row in res:
        writer.writerow(row)
sc.stop()

70106
{'pdist': '118', 'spd': '18', 'lat': '40.44067764', 'lon': '-80.00022125', 'eta': '27', 'bias': 1, 'sin_day_of_week': 0.43388373911755823, 'cos_day_of_week': -0.900968867902419, 'sin_hour_of_day': -0.8660254037844384, 'cos_hour_of_day': -0.5000000000000004, 'sin_time_of_day': -0.8890171414857364, 'cos_time_of_day': -0.45787391511695663, 'sin_hdg': 0.913545457642601, 'cos_hdg': -0.4067366430758001, 'weekday': 1, 'Braddock': 0, 'Downtown': 0, 'Greenfield Only': 0, 'McKeesport': 0, 'Murray-Waterfront': 0, 'Swissvale': 1, '61A': 1, '61B': 0, '61C': 0, '61D': 0}
dict_keys(['pdist', 'spd', 'lat', 'lon', 'eta', 'bias', 'sin_day_of_week', 'cos_day_of_week', 'sin_hour_of_day', 'cos_hour_of_day', 'sin_time_of_day', 'cos_time_of_day', 'sin_hdg', 'cos_hdg', 'weekday', 'Braddock', 'Downtown', 'Greenfield Only', 'McKeesport', 'Murray-Waterfront', 'Swissvale', '61A', '61B', '61C', '61D'])


An overview of the generated *feature.csv* file:![https://raw.githubusercontent.com/Albertle/15688_Tutorial/master/figure/Fourth_Example_2.jpeg](https://raw.githubusercontent.com/Albertle/15688_Tutorial/master/figure/Fourth_Example_2.jpeg)

In this way, we transform an ordinary program into a distributed Spark program. And if you run the above code in a real cluster, different partitions of the task will be executed in parallel on different machines to utilize the resources and speed up computation.

### Where to go from here

Congratulation! Now you are equipped with enough knowledge about Spark programming framework and its core ideas. If you want to learn more, here are some materials I highly recommended you to look at:
- [Official RDD Programming Guide](https://spark.apache.org/docs/latest/rdd-programming-guide.html): provide much more detailed information on RDD programming.
- [Pyspark API](https://spark.apache.org/docs/latest/api/python/pyspark.html)
- [MapReduce: Simplified Data Processing on Large Clusters](https://www.usenix.org/legacy/events/osdi04/tech/full_papers/dean/dean.pdf): Google's paper on MapReduce framework. Since Spark is an implementation of MapReduce, it's helpful for you to understand programming framework of Spark.
- [Spark: Cluster Computing with Working Sets](https://www.usenix.org/legacy/event/hotcloud10/tech/full_papers/Zaharia.pdf): Paper published by Spark designer, more detailed design aspect of Spark is demonstrated.